In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_avccmdm412',
    proc_date,
    'temp_sap_avccmdm412'
)

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm412
USING DELTA TBLPROPERTIES (
    delta.autoOptimize.optimizeWrite = true,
    delta.autoOptimize.autoCompact = true
)
AS
SELECT 
    bbynr,
    ersda AS created_on,
    ernam AS created_by,
    laeda AS date_of_last_change,
    aenam AS changed_by,
    zhapzeit_start AS happy_hour_start,
    zhapzeit_end AS happy_hour_end,
    zmonday AS zmonday,
    ztuesday AS ztuesday,
    zwednesday AS zwednesday,
    zthursday AS zthursday,
    zfriday AS zfriday,
    zsaturday AS zsaturday,
    zsunday AS zsunday,
    zmember AS member_only,
    custyp AS customer_type,
    zprior AS promotion_priority,
    zstock AS cash_flow_statement,
    limcus AS limit_customer,
    zccno AS credit_card_number,
    zbank AS bank,
    zcard_type AS card_type,
    zdate AS date,
    zcombination AS discount_origin_price,
    zold_promo AS old_promotion_id,
    zbbtype AS bonus_buy_note,
    zpromo_vendor AS vendor_promotion,
    zpromo_ven_per AS pct_shared_by_vendor,
    zaktnr AS vendor_promotion_id,
    zapplyadr AS apply_pnl,
    zfont_th AS pct_font_th,
    zfont_km AS pct_font_km,
    zclaim_gt AS value_claimback,
    zclaim_ht AS form_claimback,
    buy_prqcur AS prerequisite_currency,
    proc_date,
    file_creation_ts
FROM temp_sap_avccmdm412
""")